In [2]:
from sklearn.metrics import f1_score
import ast


data=[]
prediction_new = []
label_new = []

# Read the file and extract the prediction and label columns
line = "17625	['O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', '[CLS]', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O']	['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-A2', 'I-A2', 'O', 'B-A1', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']"


uid, prediction, label = line.strip().split("\t")
data.append((prediction.strip("[]"), label.strip("[]")))

print(f"data: type: {type(data)}")
print(f"prediction: {prediction}")
print(f"label: {label}")
# convert list to string
predictions_data = [d[0] for d in data]
labels_data = [d[1] for d in data]

lineTok = ast.literal_eval(label)
lineLab = ast.literal_eval(prediction)

print(f"lineTok: {lineTok}")
print(f"lineLab: {lineLab}")
for (Tok, Lab) in zip(lineTok, lineLab):
    
    if Lab in ['[CLS]','[SEP]', 'X']: # replace non-text tokens with O. These will not be evaluated.
        prediction_new.append('O')
        label_new.append('O')
        continue
    if(Lab == "B-V"):
        prediction_new.append("V")
    else:
        prediction_new.append(Lab)
        label_new.append(Tok)
        
print(f"prediction_new: {prediction_new}")
print(f"label_new: {label_new}")

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import recall_score as class_recall_score

new1 = [label_new]
print(f"new1: {new1}")
new2 = [prediction_new]

result_f1 = f1_score(label_new, prediction_new,average="micro")

print(f"result_f1: {result_f1}")

data: type: <class 'list'>
prediction: ['O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', '[CLS]', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O']
label: ['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-A2', 'I-A2', 'O', 'B-A1', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']
lineTok: ['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-A2', 'I-A2', 'O', 'B-A1', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']
lineLab: ['O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', '[CLS]', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O']
prediction_new: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
label_new: ['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-A2', 'I-A2', 'O', 'O', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']
new1: [['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-A2', 'I-A2', 'O', 'O', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']]
result_f1: 0.75


In [60]:
data = []
labels = []
predictions = []

filePath = "./output/conllsrl_test_predictions_29.tsv"
with open(filePath, "r") as f:
    for line in f:
        # delete the first line
        if "uid" in line:
            continue
        uid, prediction, label = line.strip().split("\t")
        labels.append(label)
        predictions.append(prediction)
        data.append((prediction.strip("[]"), label.strip("[]")))

print("prediction: ", type(data))

prediction:  <class 'list'>


In [3]:
data = []
labels = []
predictions = []

filePath = "./output/conllsrl_test_predictions_29.tsv"
with open(filePath, "r") as f:
    for line in f:
        # delete the first line
        if "uid" in line:
            continue
        uid, prediction, label = line.strip().split("\t")
        labels.append(label)
        predictions.append(prediction)
        data.append((prediction.strip("[]"), label.strip("[]")))

print("prediction: ", type(data))
# apply this conversion to all the elements in the list
lineToks = [ast.literal_eval(label) for label in labels]
lineLabs = [ast.literal_eval(prediction) for prediction in predictions]

trueLabels = [] # labels
predictLabels = [] # predictions

for lineTok, lineLab in zip(lineToks, lineLabs):
    if lineLab in ['[CLS]','[SEP]', 'X']: # replace non-text tokens with O. These will not be evaluated.
        predictLabels.append('O')
        trueLabels.append('O')
        continue
    if(lineLab == "B-V"):
        predictLabels.append("V")
    else:
        predictLabels.append(lineLab)
        trueLabels.append(lineTok) 
 
# flatten the list of lists into a single list
trueLabels = [item for sublist in trueLabels for item in sublist]
predictLabels = [item for sublist in predictLabels for item in sublist]
       
result_f1 = f1_score(trueLabels, predictLabels,average="micro")

print(f"result_f1: {result_f1}")

prediction:  <class 'list'>
result_f1: 0.761921602045164


----


In [12]:
import torch

from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2', do_lower_case=True,truncation=True)


In [7]:

text = "A G-to-A transition at the first nucleotide of intron 2 of patient 1 abolished normal splicing."

marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

['[CLS]', 'a', 'g', '-', 'to', '-', 'a', 'transition', 'at', 'the', 'first', 'n', '##uc', '##leo', '##tide', 'of', 'intro', '##n', '2', 'of', 'patient', '1', 'abolished', 'normal', 's', '##p', '##licing', '.', '[SEP]']


In [ ]:
# print content of the biobert vocabulary
list(tokenizer.vocab.keys())[5000:5020]

In [8]:
# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
a               170
g               176
-               118
to            1,106
-               118
a               170
transition    6,468
at            1,120
the           1,103
first         1,148
n               183
##uc         21,977
##leo        26,918
##tide       23,767
of            1,104
intro        27,553
##n           1,179
2               123
of            1,104
patient       5,351
1               122
abolished     8,632
normal        2,999
s               188
##p           1,643
##licing     22,548
.               119
[SEP]           102


In [9]:
# Mark each of the 29 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [15]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('dmis-lab/biobert-base-cased-v1.2',
                                  output_hidden_states = True # Whether the model returns all hidden-states.
                                  )
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [16]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [17]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 29
Number of hidden units: 768


In [18]:
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())

      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 29, 768])


In [19]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 29, 768])

In [20]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 29, 768])

In [21]:
# Swap dimensions 0 and 1, switch around the “layers” and “tokens” dimensions
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([29, 13, 768])